# GAN Experimentation and Results Analysis - Interactive Notebook

This notebook provides an interactive way to understand, test, and visualize components of the GAN experimentation pipeline (`test_models.py`, `metrics_calculator.py`, `plot_results.py`).

**Instructions:**
1.  Ensure you have the necessary Python scripts in the same directory or accessible in your Python path.
2.  Modify paths and parameters in the code cells as needed.
3.  Execute cells sequentially or as desired to explore different functionalities.

## Setup: Import Libraries and Define Common Parameters

In [ ]:
# --- Essential Imports ---
import os
import subprocess
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- Potentially from your scripts (add as needed) ---
# from metrics_calculator import calculate_all_metrics, calculate_fid_metric # etc.
# from plot_results import plot_experiment_results # etc.

# --- Common Parameters (Modify these) ---
BASE_DIR = "." # Assuming scripts are in the current directory
DATAROOT = "./datasets/your_dataset/" # Path to your dataset
MODEL_TYPE = "pix2pix" # or "cycle_gan"
DIRECTION = "AtoB" # or "BtoA"
GPU_ID_STR = "0" # GPU to use, or "-1" for CPU
NUM_TEST_IMAGES = 50 # Number of images to test

CHECKPOINTS_BASE_DIR = os.path.join(BASE_DIR, "checkpoints")
RESULTS_BASE_DIR = os.path.join(BASE_DIR, "results")
METRICS_DETAILS_DIR = os.path.join(BASE_DIR, "metrics_details")
PLOTS_OUTPUT_DIR = os.path.join(BASE_DIR, "plots_notebook")
EXPERIMENT_LOG_CSV = os.path.join(BASE_DIR, "experiment_log_notebook.csv")

# Create output directories if they don't exist
os.makedirs(METRICS_DETAILS_DIR, exist_ok=True)
os.makedirs(PLOTS_OUTPUT_DIR, exist_ok=True)

print(f"Base directory: {os.path.abspath(BASE_DIR)}")
print(f"Plots will be saved to: {os.path.abspath(PLOTS_OUTPUT_DIR)}")

## `test_models.py`: Defining Experiment Configurations

Here, we can define or load example experiment configurations similar to how `test_models.py` does.

In [ ]:
example_experiment_config = {
    'id': 'NB_Test01',
    'lr': 0.0002,
    'batch_size': 1,
    'n_epochs': 1, # Keep low for quick testing
    'n_epochs_decay': 0,
    'netG': 'unet_256',
    'netD': 'basic',
    'norm': 'batch',
    'preprocess': 'resize_and_crop',
    'load_size': 256,
    'crop_size': 256,
    'no_flip': False,
    'model_type': MODEL_TYPE,
    'direction': DIRECTION,
    'dataroot': DATAROOT,
    'gpu_ids': GPU_ID_STR
}

print("Example Experiment Configuration:")
print(json.dumps(example_experiment_config, indent=2))

## `test_models.py`: Simulating an Experiment Run (Command Generation)

This cell demonstrates how commands might be generated. To actually run them, you'd typically use `subprocess.run()` as in `test_models.py`. For safety in a notebook, we'll just print the commands.

In [ ]:
def generate_train_command(config):
    cmd = [
        "python", "train.py",
        "--dataroot", str(config['dataroot']),
        "--name", f"{config['model_type']}_{config['id']}",
        "--model", str(config['model_type']),
        "--direction", str(config['direction']),
        "--gpu_ids", str(config['gpu_ids']),
        "--lr", str(config['lr']),
        "--batch_size", str(config['batch_size']),
        "--n_epochs", str(config['n_epochs']),
        "--n_epochs_decay", str(config['n_epochs_decay']),
        "--netG", str(config['netG']),
        "--netD", str(config['netD']),
        "--norm", str(config['norm']),
        "--preprocess", str(config['preprocess']),
        "--load_size", str(config['load_size']),
        "--crop_size", str(config['crop_size']),
        "--checkpoints_dir", CHECKPOINTS_BASE_DIR
    ]
    if config.get('no_flip'):
        cmd.append("--no_flip")
    return cmd

def generate_test_command(config):
    cmd = [
        "python", "test.py",
        "--dataroot", str(config['dataroot']),
        "--name", f"{config['model_type']}_{config['id']}",
        "--model", str(config['model_type']),
        "--direction", str(config['direction']),
        "--gpu_ids", str(config['gpu_ids']),
        "--netG", str(config['netG']),
        "--norm", str(config['norm']),
        "--preprocess", str(config['preprocess']),
        "--load_size", str(config['load_size']),
        "--crop_size", str(config['crop_size']),
        "--results_dir", RESULTS_BASE_DIR,
        "--checkpoints_dir", CHECKPOINTS_BASE_DIR,
        "--num_test", str(NUM_TEST_IMAGES),
        "--eval" # Important for consistent naming for metrics
    ]
    return cmd

train_cmd_str = " ".join(generate_train_command(example_experiment_config))
test_cmd_str = " ".join(generate_test_command(example_experiment_config))

print("--- Example Training Command ---")
print(train_cmd_str)
print("\n--- Example Testing Command ---")
print(test_cmd_str)

# To actually run (use with caution, ensure paths and scripts are correct):
# print("\n--- Running Training (Example - Will take time) ---")
# subprocess.run(generate_train_command(example_experiment_config), check=True)
# print("\n--- Running Testing (Example) ---")
# subprocess.run(generate_test_command(example_experiment_config), check=True)

## `test_models.py`: Logging Experiment Results (CSV and JSON)

Simulate logging results to a CSV and saving MSE scores to JSON.

In [ ]:
# Simulate some results after an experiment run
mock_results = {
    'exp_id': example_experiment_config['id'],
    'full_exp_name': f"{example_experiment_config['model_type']}_{example_experiment_config['id']}",
    'lr': example_experiment_config['lr'],
    'batch_size': example_experiment_config['batch_size'],
    # ... other params from config ...
    'avg_psnr': 25.5,
    'avg_ssim': 0.85,
    'fid': 50.1,
    'avg_lpips': 0.1,
    'avg_mse': 0.01
}

mock_mse_scores = np.random.rand(NUM_TEST_IMAGES) * 0.05 # Example MSE scores

# --- Log to CSV ---
df_log = pd.DataFrame([mock_results])
if not os.path.exists(EXPERIMENT_LOG_CSV):
    df_log.to_csv(EXPERIMENT_LOG_CSV, index=False)
else:
    df_log.to_csv(EXPERIMENT_LOG_CSV, mode='a', header=False, index=False)
print(f"Appended results to {EXPERIMENT_LOG_CSV}")
display(pd.read_csv(EXPERIMENT_LOG_CSV).tail())

# --- Save MSE scores to JSON ---
mse_json_path = os.path.join(METRICS_DETAILS_DIR, f"{mock_results['exp_id']}_mse_scores.json")
with open(mse_json_path, 'w') as f:
    json.dump(mock_mse_scores.tolist(), f)
print(f"Saved MSE scores to {mse_json_path}")

## `metrics_calculator.py`: Calculating Metrics

Here you would call functions from your `metrics_calculator.py` script. We'll use placeholders if the script/dependencies are not directly available in the notebook's environment.

In [ ]:
def calculate_psnr_ssim_mse_for_images_notebook_example(gen_img_array, real_img_array):
    """Placeholder: Replace with actual skimage.metrics calls."""
    # Assuming images are numpy arrays, normalized [0,1]
    mse = np.mean((real_img_array - gen_img_array) ** 2)
    if mse == 0:
        psnr = float('inf')
    else:
        data_range = 1.0
        psnr = 20 * np.log10(data_range / np.sqrt(mse))
    ssim = 0.9 # Dummy value
    print(f"Placeholder PSNR: {psnr:.2f}, SSIM: {ssim:.2f}, MSE: {mse:.4f}")
    return psnr, ssim, mse

# Example: Create dummy image data (replace with actual image loading)
dummy_real_img = np.random.rand(256, 256, 3)
dummy_gen_img = np.clip(dummy_real_img + np.random.normal(0, 0.1, (256,256,3)), 0, 1)

calculate_psnr_ssim_mse_for_images_notebook_example(dummy_gen_img, dummy_real_img)

# --- For LPIPS and FID, you'd typically call your script's functions ---
# output_images_dir_for_metrics = os.path.join(RESULTS_BASE_DIR, f"{MODEL_TYPE}_{example_experiment_config['id']}", "test_latest", "images")
# if os.path.exists(output_images_dir_for_metrics):
#     avg_p, avg_s, avg_l, avg_m, all_m = calculate_all_metrics(output_images_dir_for_metrics, direction=DIRECTION)
#     print(f"Calculated All Metrics: PSNR={avg_p:.2f}, SSIM={avg_s:.2f}, LPIPS={avg_l:.3f}, MSE={avg_m:.4f}")
#     fid_score = calculate_fid_metric(output_images_dir_for_metrics, direction=DIRECTION)
#     print(f"Calculated FID: {fid_score:.2f}")
# else:
#     print(f"Image directory for metrics not found: {output_images_dir_for_metrics}")
print("\nLPIPS and FID calculation would require actual images and dependencies.")
print("Uncomment and adapt above lines if you have run an experiment and have output images.")

## `plot_results.py`: Loading Data and Generating Plots

Load the experiment log and metrics details to generate visualizations.

In [ ]:
# --- Load Experiment Log ---
if os.path.exists(EXPERIMENT_LOG_CSV):
    df_results = pd.read_csv(EXPERIMENT_LOG_CSV)
    print("Loaded experiment log:")
    display(df_results.head())
else:
    print(f"Experiment log {EXPERIMENT_LOG_CSV} not found. Run previous cells to create it.")
    df_results = pd.DataFrame() # Empty dataframe

# --- Load MSE scores for a specific experiment (if available) ---
exp_id_to_plot = example_experiment_config['id'] # From earlier cell
mse_scores_path_to_load = os.path.join(METRICS_DETAILS_DIR, f"{exp_id_to_plot}_mse_scores.json")
loaded_mse_scores = []
if os.path.exists(mse_scores_path_to_load):
    with open(mse_scores_path_to_load, 'r') as f:
        loaded_mse_scores = json.load(f)
    print(f"\nLoaded {len(loaded_mse_scores)} MSE scores for experiment {exp_id_to_plot}.")
else:
    print(f"\nMSE scores file not found: {mse_scores_path_to_load}")
    loaded_mse_scores = mock_mse_scores.tolist() # Use mock if not found for plotting demo
    print(f"Using mock MSE scores for plotting demo.")

### Plotting: Bar Plot for Overall Metric Comparison

In [ ]:
if not df_results.empty:
    # Ensure numeric types for plotting
    metrics_to_plot = ['avg_psnr', 'avg_ssim', 'fid', 'avg_lpips', 'avg_mse']
    for metric in metrics_to_plot:
        if metric in df_results.columns:
            df_results[metric] = pd.to_numeric(df_results[metric], errors='coerce')

    plt.figure(figsize=(10, 6))
    # Plotting avg_mse as an example
    if 'avg_mse' in df_results.columns and not df_results['avg_mse'].isnull().all():
        sns.barplot(x='exp_id', y='avg_mse', data=df_results)
        plt.title('Average MSE Comparison Across Experiments')
        plt.xticks(rotation=45, ha='right')
        plt.ylabel('Average MSE')
        plt.xlabel('Experiment ID')
        plt.tight_layout()
        plot_save_path = os.path.join(PLOTS_OUTPUT_DIR, "bar_avg_mse_comparison.png")
        plt.savefig(plot_save_path)
        plt.show()
        print(f"Saved bar plot to {plot_save_path}")
    else:
        print("Skipping avg_mse bar plot: No data or all NaN.")
else:
    print("DataFrame is empty, skipping bar plot.")

### Plotting: MSE Distributions (Boxplot and Histogram)

In [ ]:
if loaded_mse_scores:
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Boxplot
    axes[0].boxplot(loaded_mse_scores, vert=True)
    axes[0].set_title(f'Boxplot of MSE Scores\n{exp_id_to_plot}')
    axes[0].set_ylabel('MSE')
    axes[0].set_xticks([])

    # Histogram
    axes[1].hist(loaded_mse_scores, bins=20, edgecolor='black', alpha=0.7)
    axes[1].set_title(f'Histogram of MSE Scores\n{exp_id_to_plot}')
    axes[1].set_xlabel('MSE')
    axes[1].set_ylabel('Frequency')
    
    plt.tight_layout()
    plot_save_path = os.path.join(PLOTS_OUTPUT_DIR, f"mse_distribution_{exp_id_to_plot}.png")
    plt.savefig(plot_save_path)
    plt.show()
    print(f"Saved MSE distribution plot to {plot_save_path}")
else:
    print("No MSE scores loaded to plot distributions.")